In [7]:
import torch 
import math

if torch.cuda.is_available():
    print('Its Cudaaaaa time')
    torch.set_default_device('cuda')

In [6]:
#Hyper Paramaters

K_PULL = 1
K_PUSH = 1

In [ ]:
def pullPE(vector1 : torch.tensor, vector2 : torch.tensor):
    distance = torch.linalg.norm(vector1 - vector2)
    return (1/2) * K_PULL * (distance ** 2)

def pushPE(vector1 : torch.tensor, vector2 : torch.tensor):
    distance = torch.linalg.norm(vector1 - vector2)
    return -K_PUSH * math.log(distance)

In [ ]:
def wrapped_distance(vector1: torch.Tensor, vector2: torch.Tensor):
    """Calculates the wrapped distance between two points in a periodic space."""
    dx = torch.abs(vector1[0] - vector2[0])
    dy = torch.abs(vector1[1] - vector2[1])
    
    dx = torch.where(dx > WIDTH / 2, WIDTH - dx, dx)
    dy = torch.where(dy > HEIGHT / 2, HEIGHT - dy, dy)
    
    return torch.sqrt(dx ** 2 + dy ** 2)

In [4]:


# Check if a GPU is available
if torch.cuda.is_available():
    print("cudaaaaa")
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Create tensors on the GPU
x = torch.randn(1000, 1000, device=device)
y = torch.randn(1000, 1000, device=device)

# Perform calculations on the GPU
z = torch.matmul(x, y)  # This matrix multiplication happens on the GPU

In [ ]:
import torch
import math

# Set device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

K_PULL = 1.0  # Adjust this constant as needed for pull force
K_PUSH = 1.0  # Adjust this constant as needed for push force
DIMENSIONS = 2  # Can be set to any number of dimensions
WIDTH, HEIGHT = 1000, 1000  # Dimensions of the square

def wrapped_distance(nodes: torch.Tensor):
    """Calculates the wrapped distance matrix for a batch of nodes in a periodic space."""
    # Calculate direct differences
    dx = nodes[:, None, 0] - nodes[None, :, 0]  # Shape (N, N)
    dy = nodes[:, None, 1] - nodes[None, :, 1]  # Shape (N, N)
    
    # Wrap around for each dimension
    dx = torch.where(torch.abs(dx) > WIDTH / 2, WIDTH - torch.abs(dx), torch.abs(dx))
    dy = torch.where(torch.abs(dy) > HEIGHT / 2, HEIGHT - torch.abs(dy), torch.abs(dy))
    
    # Compute the Euclidean distance
    return torch.sqrt(dx ** 2 + dy ** 2)

def pullPE(distance_matrix: torch.Tensor):
    """Calculates the potential energy for attracting connected nodes."""
    return (1/2) * K_PULL * (distance_matrix ** 2)

def pushPE(distance_matrix: torch.Tensor):
    """Calculates the potential energy for repelling unconnected nodes."""
    # Use torch.log to avoid log(0) for distances
    with torch.no_grad():
        return -K_PUSH * torch.log(distance_matrix + 1e-8)  # Adding small value to avoid log(0)

def calculateTotalEnergy(nodes: torch.Tensor, edges: list):
    """Calculates the total potential energy for the graph."""
    total_energy = torch.tensor(0.0, device=device)  # Initialize on the correct device
    distance_matrix = wrapped_distance(nodes)  # Shape (N, N)

    for i in range(len(nodes)):
        for j in range(len(nodes)):
            if i != j:
                if (i, j) in edges or (j, i) in edges:  # Check if there's an edge
                    total_energy += pullPE(distance_matrix[i, j])
                else:
                    total_energy += pushPE(distance_matrix[i, j])

    return total_energy

# Example usage
nodes = torch.tensor([
    [100.0, 100.0],
    [900.0, 100.0],
    [500.0, 900.0],
    [150.0, 850.0],
], device=device)

edges = [(0, 1), (1, 2), (2, 0)]  # Example edges connecting nodes

total_energy = calculateTotalEnergy(nodes, edges)
print("Total Energy:", total_energy.item()) 

In [ ]:
import torch
import math

# Set device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

K_PULL = 1.0  # Adjust this constant as needed for pull force
K_PUSH = 1.0  # Adjust this constant as needed for push force

# Dimensions of the space (can be set to any number)
DIMENSIONS = 3
BOUNDARIES = [1000] * DIMENSIONS  # Example boundaries for each dimension (1000 for each)

def wrapped_distance(nodes: torch.Tensor):
    """Calculates the wrapped distance matrix for a batch of nodes in a periodic space."""
    # Compute differences for all pairs of nodes using broadcasting
    diff = nodes[:, None, :] - nodes[None, :, :]  # Shape (N, N, DIMENSIONS)

    # Apply wrap-around for each dimension
    for dim in range(DIMENSIONS):
        abs_diff = torch.abs(diff[:, :, dim])  # Shape (N, N)
        abs_diff = torch.where(abs_diff > BOUNDARIES[dim] / 2, BOUNDARIES[dim] - abs_diff, abs_diff)
        diff[:, :, dim] = abs_diff  # Update the difference with wrapped distances

    # Compute the Euclidean distance across all dimensions
    return torch.sqrt(torch.sum(diff ** 2, dim=-1))  # Shape (N, N)

def pullPE(distance_matrix: torch.Tensor):
    """Calculates the potential energy for attracting connected nodes."""
    return (1/2) * K_PULL * (distance_matrix ** 2)

def pushPE(distance_matrix: torch.Tensor):
    """Calculates the potential energy for repelling unconnected nodes."""
    # Use torch.log to avoid log(0) for distances
    with torch.no_grad():
        return -K_PUSH * torch.log(distance_matrix + 1e-8)  # Adding small value to avoid log(0)

def calculateTotalEnergy(nodes: torch.Tensor, edges: list):
    """Calculates the total potential energy for the graph."""
    total_energy = torch.tensor(0.0, device=device)  # Initialize on the correct device
    distance_matrix = wrapped_distance(nodes)  # Shape (N, N)

    for i in range(len(nodes)):
        for j in range(len(nodes)):
            if i != j:
                if (i, j) in edges or (j, i) in edges:  # Check if there's an edge
                    total_energy += pullPE(distance_matrix[i, j])
                else:
                    total_energy += pushPE(distance_matrix[i, j])

    return total_energy

# Example usage
nodes = torch.tensor([
    [100.0, 100.0, 50.0],
    [900.0, 100.0, 800.0],
    [500.0, 900.0, 200.0],
    [150.0, 850.0, 100.0],
], device=device)

edges = [(0, 1), (1, 2), (2, 0)]  # Example edges connecting nodes

total_energy = calculateTotalEnergy(nodes, edges)
print("Total Energy:", total_energy.item()) 

In [11]:
nodes = torch.tensor([
    [1, 2, 3],
    [4, 5, 6],
    [9, 8, 7],
    [10, 11, 12]
])

diff = nodes[:, None, :] - nodes[None, :, :]

print(diff)

tensor([[[ 0,  0,  0],
         [-3, -3, -3],
         [-8, -6, -4],
         [-9, -9, -9]],

        [[ 3,  3,  3],
         [ 0,  0,  0],
         [-5, -3, -1],
         [-6, -6, -6]],

        [[ 8,  6,  4],
         [ 5,  3,  1],
         [ 0,  0,  0],
         [-1, -3, -5]],

        [[ 9,  9,  9],
         [ 6,  6,  6],
         [ 1,  3,  5],
         [ 0,  0,  0]]], device='cuda:0')


In [ ]:
import torch

# Create a COO sparse tensor
indices = torch.tensor([[0, 1, 1],
                        [2, 0, 2]])  # (row indices, col indices)
values = torch.tensor([3, 4, 5], dtype=torch.float32)
size = torch.Size([3, 3])

# Create sparse tensor
sparse_matrix = torch.sparse_coo_tensor(indices, values, size)

# Convert to dense for inspection
dense_matrix = sparse_matrix.to_dense()
print(dense_matrix)

# Perform matrix-vector multiplication
vector = torch.tensor([1, 2, 3], dtype=torch.float32)
result = torch.sparse.mm(sparse_matrix, vector.unsqueeze(1))
print(result)

Sparsity Threshold
Sparse matrices are most beneficial when there’s a high proportion of zero entries. If the matrix is too dense, using a sparse format could lead to worse performance due to the overhead of managing the sparse structure. As a rule of thumb:

Use sparse matrices if more than ~50-70% of the matrix elements are zeros.

Analyze density to determine if a spare represntation would be a good idea.

4. Reduce Memory Usage
In-Place Operations: When possible, use in-place operations to avoid unnecessary memory allocations, which can slow down computations.

Memory Efficient Data Types: If precision allows, use lower precision (e.g., torch.float16 or torch.float32 instead of torch.float64) to save memory and increase speed.

https://machinelearningmastery.com/calculating-derivatives-in-pytorch/

songs = {1,2}
connections = {}

for playlist in data['playlists']:
    for idx, track in enumerate(playlist['tracks']):
        
        if not (track['artist_name'] in songs):
            songs.add('artist_name')
        for i in range(idx+1,len(playlist['tracks'])):
            connection = track['artist_name'] + ":::" + playlist['tracks'][i]['artist_name']
            if connection in connections:
                connections[connection] += 1
            else:
                connections[connection]  = 1

In [ ]:
nodes = torch.tensor([[0, 0, 0], [0, 3, 0], [0, 0, 4]])
#sparse_tensor = dense_tensor.to_sparse()


diff = nodes[:, None, :] - nodes[None, :, :]

sparse_tensor = diff.to_sparse()


In [ ]:
import random
cords = [[random.random(),random.random(),random.random()] for _ in range(10000000)]
weights = [random.random() for _ in range(10000000)]

print(calculateForce(torch.tensor([cords]),torch.tensor(weights)))